In [1]:
import pandas as pd
import stanza
import os
import requests

In [ ]:
stanza.download('fr')

In [2]:
path_csv = "data/flaubert_dates-texte.csv"

In [45]:
df_text_dates = pd.read_csv(path_csv, encoding='UTF-8', sep='|')

## Faire une liste de noms de lieux

In [ ]:
nlp = stanza.Pipeline('fr')

In [49]:
def get_loc(texte) :
    liste_loc = []
    nlp_text = nlp(texte)
    for ent in nlp_text.ents : 
        if ent.type =='LOC':
            liste_loc.append(ent.text) 
    return liste_loc

In [50]:
placenames = []
for i in range(len(df_text_dates)) :
    texte = df_text_dates.loc[i, 'texte_source']
    liste_loc = get_loc(texte)
    placenames.extend(liste_loc)



In [ ]:
# Créer un csv avec les noms de lieux pour effectuer des corrections et compléter avec des noms de lieu normalisés 
data={'place_source': placenames}
df_places = pd.DataFrame(data)
df_places=df_places.drop_duplicates(ignore_index=True)
df_places.to_csv('data/placenames_stanza.csv', encoding='UTF-8', sep=';')

## Fusionnement des données

In [ ]:
df_placenames = pd.read_csv('data/PlaceNames.csv', sep=';')

In [5]:
path_csv_main = './data/data_flaubert.csv'


In [6]:
df_main = pd.read_csv(path_csv_main, encoding='UTF-8', sep='|')
df_main = df_main.map(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
#Créer 2 dfs distincts avec 2 géométries différentes (points, lignes) pour importer sur QGIS
#Créer un df avec les lignes qui correspondent uniquement aux chemins et excluent les points relais se trouvant sur chemins
df_roads = df_main[(df_main['road'] == True)]
df_roads.drop(df_roads[(df_roads['from_boat'] == True) & (df_roads['loc'] == True)].index, inplace=True) #Enlever les points vus depuis bateau

#Créer un df avec les points
df_points = df_main[(df_main['loc']== True)]



### Récupérer les coordonnées

In [2]:
#Ouvrir le fichier contenant les noms de lieu, le noms de lieu normalisés et le wiki_id qui leurs correspondent
path_placenames = 'data/PlaceNames.csv'

with open(path_placenames, encoding='UTF-8') as file :
    list_places = file.readlines()

list_places = [place.strip('\n').split(';') for place in list_places]

In [ ]:
#obtenir les coordonnées pour wiki_id
api_url = "https://www.wikidata.org/w/api.php"

def get_coords(wiki_id):
    params = {
        "action": "wbgetentities",
        "ids": wiki_id,
        "props": "claims",
        "format": "json"
    }
    
    response = requests.get(api_url, params=params)
    data = response.json()

    try:
        coordinates = data["entities"][wiki_id]["claims"]["P625"][0]["mainsnak"]["datavalue"]["value"]
        lat, lon = coordinates["latitude"], coordinates["longitude"]
        return (lat, lon)
    
    except KeyError:
        return (None, None)  # Si aucune coordonnée


In [ ]:
for lieu in list_places :
    coords = get_coords(lieu[2])
    lieu.extend(coords)

# Créer df_placenames
columns = ['placename_source', 'placename_normalized', 'wiki_id', 'lat', 'lon']
df_placenames = pd.DataFrame(data=list_places, columns=columns)

### Jointures

In [23]:
def merge_geo_df(df_source, col_name, df_placenames) :
    geo_df = df_source.merge(df_placenames, left_on=col_name, right_on='placename_source', how='left')
    geo_df.drop(columns=['placename_source'], inplace=True)
    return geo_df

In [ ]:
#Créer geo_df_points, faire une jointure avec df_placenames et renommer les nouvelles colonnes pour faire deux jointures sur deux colonnes différentes. 
geo_df_points = merge_geo_df(df_points, 'place_source', df_placenames)
geo_df_points.rename(columns={'placename_normalized':'place_source_normalized',
                             'wiki_id':'place_source_wiki_id',
                             'lat': 'lat_source', 
                             'lon': 'lon_source'}, inplace=True)

geo_df_points = merge_geo_df(geo_df_points, 'place_compare', df_placenames)
geo_df_points.rename(columns={'placename_normalized': 'place_compare_normalized', 
                               'wiki_id': 'place_compare_wiki_id', 
                               'lat': 'lat_compare', 
                               'lon': 'lon_compare'}, inplace=True)


In [ ]:
# Créer un nouveau df pour les routes avec coordonnées. 
geo_df_roads = merge_geo_df(df_roads, 'place_source_start', df_placenames)

geo_df_roads.rename(columns={'placename_normalized':'place_start_normalized',
                             'wiki_id':'place_start_wiki_id',
                             'lat': 'lat_start', 
                             'lon': 'lon_start'}, inplace=True)

geo_df_roads = merge_geo_df(geo_df_roads, 'place_source_end', df_placenames)

geo_df_roads.rename(columns={'placename_normalized':'place_end_normalized',
                             'wiki_id':'place_end_wiki_id',
                             'lat': 'lat_end', 
                             'lon': 'lon_end'}, inplace=True)

geo_df_roads = merge_geo_df(geo_df_roads, 'place_source', df_placenames)

geo_df_roads.rename(columns={'placename_normalized':'place_source_normalized',
                             'wiki_id':'place_source_wiki_id',
                             'lat': 'lat_source', 
                             'lon': 'lon_source'}, inplace=True)

geo_df_roads=merge_geo_df(geo_df_roads, 'place_compare', df_placenames)
geo_df_roads.rename(columns={'placename_normalized': 'place_compare_normalized', 
                               'wiki_id': 'place_compare_wiki_id', 
                               'lat': 'lat_compare', 
                               'lon': 'lon_compare'}, inplace=True)

### Créer une géométrie et enregistrer

In [ ]:
#Fonction pour ajouter un objet wkt pour créer une ligne
def wkt(ligne):
    if not pd.isna(ligne['lat_start']) and not pd.isna(ligne['lon_start']) and not pd.isna(ligne['lat_end']) and not pd.isna(ligne['lon_end']):
        coords = [(ligne['lon_start'], ligne['lat_start'])]
        
        # Ajouter points relais sur la route
        if ligne['loc']== True :
            coords.append((ligne['lon_source'], ligne['lat_source']))
        coords.append((ligne['lon_end'], ligne['lat_end']))
        
        return f"LINESTRING ({', '.join(f'{lon} {lat}' for lon, lat in coords)})"
    else:
        return None  # si pas de coordonnées


In [ ]:
geo_df_roads['wkt'] = geo_df_roads.apply(wkt, axis=1)

In [ ]:
# créer tsv
def df_2_tsv(df, filename):
    os.makedirs('QGIS/data_geo', exist_ok=True)
    path = os.path.join('qgis/data_geo', filename + '.tsv')
    df.to_csv(path, encoding='UTF-8', sep='\t')

In [ ]:
df_2_tsv(geo_df_points, 'points')
df_2_tsv(geo_df_roads, 'routes')